In [1]:
%autosave 0
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Autosave disabled


# <a href="https://en.wikipedia.org/wiki/Unification_(computer_science)">Unification</a>

This notebook implements the algorithm of Martelli and Montanari for the unification of terms.

## Utility Functions

In [2]:
import folParser as fp

In [3]:
def parseTerm(s):
    parser = fp.LogicParser(s)
    return parser.parse()

The method $\texttt{apply}(t, \sigma)$ takes an object $t$ and a substitution $\sigma$ and computes the $t\sigma$, i.e. it <em style="color:blue;">applies</em> the substitution $\sigma$ to $t$.  The object $t$ is either a term or a set of terms.  The substitution $\sigma$ is represented as a dictionary.  Assume that $\sigma = \bigl[ x_1 \mapsto t_1, \cdots, x_n \mapsto t_n \bigr]$.  Then $t\sigma$ is defined by induction on $t$:
<ol>
<li>If $t$ is a variable, there are two cases when defining $t\sigma$:</li>
  <ol>
  <li>$t = x_i$ for an $i\in\{1,\cdots,n\}$.  Then we define  
      $$ x_i\sigma := t_i. $$
      </li>
  <li>$t = y$ where $y\in\mathcal{V}$, but $y \not\in \{x_1,\cdots,x_n\}$. Then we define   
      $$ y\sigma := y.$$</li>
  </ol>
<li>Otherwise we must have $t = f(s_1,\cdots,s_m)$. Then we define: 
      $$f(s_1, \cdots, s_m)\sigma := f(s_1\sigma, \cdots, s_m\sigma). $$
      </li>
</ol>

In [4]:
def apply(t, σ):
    "Apply the substitution σ to the term t."
    if isinstance(t, frozenset):                     # t is a clause
        return frozenset({ apply(c, σ) for c in t })
    if isinstance(t, set):                           # t is a set of equations
        return { apply(c, σ) for c in t }
    if isinstance(t, str):                           # t is a variable
        if t in σ:
            return σ[t]
        else:
            return t
    else: # t must be a term, literal, or equation
        f  = t[0]
        ts = t[1:]
        return (f,) + tuple(apply(s, σ) for s in ts)

In [5]:
s1 = parseTerm('G(z)')
s2 = parseTerm('H(u, v)')
σ = { 'x': s1, 'y': s2 }
σ

{'x': ('G', 'z'), 'y': ('H', 'u', 'v')}

In [6]:
t = parseTerm('F(x,H(y,x),G(z))')
t

('F', 'x', ('H', 'y', 'x'), ('G', 'z'))

In [7]:
apply(t, σ)

('F', ('G', 'z'), ('H', ('H', 'u', 'v'), ('G', 'z')), ('G', 'z'))

If  $\sigma = \big[ x_1 \mapsto s_1, \cdots, x_m \mapsto s_m \big]$ and
$\tau = \big[ y_1 \mapsto t_1, \cdots, y_n \mapsto t_n \big]$ 
are two substitutions that are <em style="color:blue;">non-overlapping</em>, i.e. such that $\texttt{dom}(\sigma) \cap \texttt{dom}(\tau) = \{\}$ holds,
then we define the <em style="color:blue;">composition</em> $\sigma\tau$ of $\sigma$ and $\tau$ as follows:
$$\sigma\tau := \big[ x_1 \mapsto s_1\tau, \cdots, x_m \mapsto s_m\tau,\; y_1 \mapsto t_1, \cdots, y_n \mapsto t_n \big]$$
The function $\texttt{compose}(\sigma, \tau)$ takes two non-overlapping substitutions and computes the composition $\sigma\tau$.

In [8]:
def compose(σ, τ):
    Result = { x: apply(s, τ) for (x, s) in σ.items() }
    Result.update(τ)
    return Result

In [9]:
τ = { 'z': s1, 'u': s2 }

In [10]:
compose(σ, τ)

{'u': ('H', 'u', 'v'),
 'x': ('G', ('G', 'z')),
 'y': ('H', ('H', 'u', 'v'), 'v'),
 'z': ('G', 'z')}

The function $\texttt{occurs}(x, t)$ checks whether the variable $x$ occurs in the term $t$.

In [11]:
def occurs(x, t):
    if x == t:
        return True
    if isinstance(t, str): # t must be a variable different from x
        return False
    return any(occurs(x, arg) for arg in t[1:])

In [12]:
t

('F', 'x', ('H', 'y', 'x'), ('G', 'z'))

In [13]:
occurs('u', t)

False

In [14]:
occurs('x', t)

True

## The Algorithm of Martelli and Montanari

The rules of Martelli and Montanari that can be used to solve a system of syntactical equations are as follows:
<ol>
<li> If $y\in\mathcal{V}$ is a variable that does <b style="color:red;">not</b> occur in the term $t$,
     then we perform the following reduction: 
     $$ \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle \quad\leadsto \quad 
         \Big\langle E[y \mapsto t], \sigma\big[ y \mapsto t \big] \Big\rangle 
     $$
</li>      
<li> If the variable $y$ occurs in the term $t$, then the system of syntactical equations
     $E \cup \big\{ y \doteq t \big\}$ is not solvable:

     $$ \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle\;\leadsto\; \texttt{None} \quad
        \mbox{if $x \in \textrm{Var}(t)$ and $y \not=t$.}$$
</li>
<li> If $y\in\mathcal{V}$ is a variable and $t$ is no variable, then we use the following rule:
     $$ \Big\langle E \cup \big\{ t \doteq y \big\}, \sigma \Big\rangle \quad\leadsto \quad 
         \Big\langle E \cup \big\{ y \doteq t \big\}, \sigma \Big\rangle.
     $$   
</li>
<li> Trivial syntactical equations of variables can be dropped:
     $$ \Big\langle E \cup \big\{ x \doteq x \big\}, \sigma \Big\rangle \quad\leadsto \quad
         \Big\langle E, \sigma \Big\rangle.
     $$   
</li>
<li> If $f$ is an $n$-ary function symbol, then we have: 
     $$ \Big\langle E \cup \big\{ f(s_1,\cdots,s_n) \doteq f(t_1,\cdots,t_n) \big\}, \sigma \Big\rangle 
         \;\leadsto\; 
         \Big\langle E \cup \big\{ s_1 \doteq t_1, \cdots, s_n \doteq t_n\}, \sigma \Big\rangle.
     $$   
</li>
<li> The system of syntactical equations $E \cup \big\{ f(s_1,\cdots,s_m) \doteq g(t_1,\cdots,t_n) \big\}$
     has <b style="color:red;">no</b> solution if the function symbols $f$ and $g$ are different:
     $$ \Big\langle E \cup \big\{ f(s_1,\cdots,s_m) \doteq g(t_1,\cdots,t_n) \big\},
      \sigma \Big\rangle \;\leadsto\; \texttt{None} \qquad \mbox{if $f \not= g$}.
     $$
</ol>


Given two terms $s$ and $t$, the function $\texttt{unify}(s, t)$ computes the <em style="color:blue;">most general unifier</em> of $s$ and $t$.

In [18]:
def unify(s, t):
    return solve({('≐', s, t)}, {})

Given a set of <em style="color:blue;">syntactical equations</em> $E$ and a substitution $\sigma$, the function $\texttt{solve}(E, \sigma)$ uses the rules of Martelli and Montanari to solve $E$.

In [19]:
def solve(E, σ):
    while E != set():
        print(E, σ)
        _, s, t = E.pop()
        if s == t:
            continue
        if isinstance(s, str): # s is a variable
            if occurs(s, t):
                return None
            else:
                apply(E, { s: t })
                σ = compose(σ, { s: t })
        elif isinstance(t, str): # t is a variable, but s is not
            E.add(('≐', t, s))
        else:
            f    , g     = s[0]      , t[0]
            sArgs, tArgs = s[1:]     , t[1:]
            m    , n     = len(sArgs), len(tArgs)
            if f != g or m != n:
                return None
            else:
                E |= { ('≐', sArgs[i], tArgs[i]) for i in range(m) }
    return σ

In [20]:
t1 = parseTerm('P(x1,F(x4))')
t2 = parseTerm('P(x2,x3)')
μ = unify(t1, t2)
μ

{('≐', ('P', 'x1', ('F', 'x4')), ('P', 'x2', 'x3'))} {}
{('≐', ('F', 'x4'), 'x3'), ('≐', 'x1', 'x2')} {}
{('≐', ('F', 'x4'), 'x3')} {'x1': 'x2'}
{('≐', 'x3', ('F', 'x4'))} {'x1': 'x2'}


{'x1': 'x2', 'x3': ('F', 'x4')}